In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
import statsmodels.api as sm

from sklearn.preprocessing import PolynomialFeatures
from plotly.subplots import make_subplots
pio.renderers.default = 'iframe'
# Set notebook mode to work in offline
pyo.init_notebook_mode()

data = pd.read_csv("/home/jessica/reading-kiln-termostat/data/dados_queimas.csv")
data['index'] = data.index

/tmp/ipykernel_4212/2403260940.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def create_curve_array(data_alpha):
    curves = []
    T_spacing = [1160, 1180, 1200]
    for index, row in data_alpha.iterrows():
        row = row.astype(float)
        curve_array = []
        curve_array = np.append(curve_array, segment_array(T_spacing[0], T_spacing[1], row.alpha_3))
        curve_array = np.append(curve_array, segment_array(T_spacing[1], T_spacing[2], row.alpha_4))
        curve_array = np.append(curve_array, segment_array(T_spacing[2], row['temp final'], row.alpha_5))
        curve_array = np.append(curve_array, [row['temp final']] * row.patamar.astype(int))
        curves.append(curve_array)
    return curves

def segment_array(initial_temp, final_temp, alpha):
    r = np.floor(np.arange(initial_temp, np.floor(final_temp), alpha))
    if r[-1] == np.floor(final_temp):
         return r
    else:
        return np.append(r, np.floor(final_temp))

In [3]:
exp_data = data.filter(regex='alpha|index|temp final|patamar|cone_meio')
exp_data['index'] = exp_data.reset_index().index
exp_data = exp_data.dropna()
for column in exp_data.columns:
    exp_data = exp_data.drop(exp_data[exp_data[column].astype(str).str.contains('-')].index)
    
    
exp_data = exp_data.astype(float)

/tmp/ipykernel_5396/917373620.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
cone_dict = {
    "rampa" : [150, 60, 40, 30, 15],
    "temp_final": [1240, 1220, 1210, 1205, 1190]
}
tab_data = pd.DataFrame(cone_dict)
tab_data['alpha'] = tab_data['rampa'] / 60
tab_data['patamar'] = 0

curves_std = []
for index, row in tab_data.iterrows():
    row = row.astype(float)
    curve = []
    curve = np.append(curve, segment_array(1160, row['temp_final'] + 0.5, row.alpha))
    curves_std.append(curve)
    
tab_data['sinal'] = curves_std

In [5]:
energia = []
for curve in curves_std:
    energia = np.append(energia,np.sum(curve))
    
tab_data['energia_SUM'] =  energia

In [6]:
curves_exp = create_curve_array(exp_data)
energia = []

for curve in curves_exp:
    energia = np.append(energia,np.sum(curve))
    
exp_data['energia_SUM'] =  energia
exp_data['sinal'] = curves_exp

In [7]:
#Plotar curvas e energia de curvas
fig = go.Figure()
tempo = np.arange(0, 85, 1)

i = 0
for curve in curves_exp:
    fig.add_scatter(x = tempo[:len(curve)], y=curve, name = "curva " + str(i), mode = 'lines')
    i+=1
    
fig.update_layout(title = "Comportamento curvas experimentais",
                 xaxis_title = 'tempo (min)',
                 yaxis_title = 'Temperatura °C')
fig

In [8]:
tab_data

,rampa,temp_final,alpha,patamar,sinal,energia_SUM
0,150,1240,2.500000,0,"[1160.0, 1162.0, 1165.0, 1167.0, 1170.0, 1172....",39592.0
1,60,1220,1.000000,0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",72590.0
2,40,1210,0.666667,0,"[1160.0, 1160.0, 1161.0, 1162.0, 1162.0, 1163....",90035.0
3,30,1205,0.500000,0,"[1160.0, 1160.0, 1161.0, 1161.0, 1162.0, 1162....",107585.0
4,15,1190,0.250000,0,"[1160.0, 1160.0, 1160.0, 1160.0, 1161.0, 1161....",142130.0


In [9]:
exp_data.sinal

1     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1166....
2     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....
3     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....
5     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....
6     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....
7     [1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....
12    [1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164....
13    [1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164....
14    [1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164....
Name: sinal, dtype: object

In [14]:
#Plotar curvas e energia de curvas
fig = go.Figure()
tempo = np.arange(0, 170, 1)
alphas = ['0.25', '0.5', '0.67', '1', '2.5']
for curve, alpha in zip(curves_std, alphas[::-1]):
    fig.add_scatter(x = tempo[:len(curve)], y=curve, name = alpha, mode = 'lines')

fig.update_layout(title =  'Comportamento Curvas Tabeladas a partir de 1160°C até Temperatura Final',
                 xaxis_title = 'tempo (min)',
                 yaxis_title = 'Temperatura (°C)'
                 )
fig

In [15]:
fig = go.Figure()
energia = [np.sum(curve) / 1000 for curve in curves_std]
tab_data['energia'] = energia
tab_data['alpha_s'] = tab_data['alpha'].astype(str)
fig = px.bar(tab_data, x='alpha_s', y='energia', color = 'alpha_s')

fig.update_layout(title = 'Heatwork (f1) - Integral da Temperatura dt para curvas tabeladas',
                 xaxis_title = 'alpha °C/min',
                  yaxis_title = 'Heatwork f1 (°C*min)'           
                 )
fig

In [16]:
error_df = pd.DataFrame(columns = ['nome', 'funcao', 'H', 'RMSE', 'MAE'])
error_df

,nome,funcao,H,RMSE,MAE


In [17]:
def energy_sum(signal):
    return np.sum(signal)

def energy_delta_sum(signal):
    return np.sum([s - 1160 for s in signal])

def energy_square(signal):
    return np.sum(np.square(signal))

def energy_delta_square(signal):
    return np.sum([((s - 1160) ** 2) for s in signal])

def energy_exp(signal):
    return np.sum(np.log(signal))

def energy_delta_exp(signal):
    return np.sum(np.sum([np.log(s - 1159) for s in signal]))

In [18]:
functions = [energy_sum, energy_delta_sum, energy_delta_square, energy_exp, energy_delta_exp]

lines = []
for function in functions:
    H = []
    for curve in curves_std:
        h = function(curve)
        H.append(h)
    mean = np.mean(H)
    mape = np.mean([abs(mean - h)/mean for h in H])
    lines.append([str(function.__name__), H, mape])
lines

[['energy_sum',
  [39592.0, 72590.0, 90035.0, 107585.0, 142130.0],
  0.3050998822831753],
 ['energy_delta_sum',
  [1312.0, 1830.0, 1875.0, 2025.0, 1770.0],
  0.10222423967317291],
 ['energy_delta_square',
  [70864.0, 73810.0, 62525.0, 60765.0, 35120.0],
  0.16824906626545771],
 ['energy_exp',
  [233.95936970836294,
   431.9775456486774,
   537.8629560859952,
   643.8354894842348,
   855.3101863744521],
  0.3092681296189955],
 ['energy_delta_exp',
  [111.97826449091356,
   192.7390472878449,
   225.55507479929125,
   262.07650867474354,
   302.06693259980585],
  0.24314162126719693]]

In [19]:
m1 = pd.DataFrame(lines, columns = ['funcao', 'H', 'MAPE'])
m1['H'] = m1['H'].apply(np.round)
m1

,funcao,H,MAPE
0,energy_sum,"[39592.0, 72590.0, 90035.0, 107585.0, 142130.0]",0.305100
1,energy_delta_sum,"[1312.0, 1830.0, 1875.0, 2025.0, 1770.0]",0.102224
2,energy_delta_square,"[70864.0, 73810.0, 62525.0, 60765.0, 35120.0]",0.168249
3,energy_exp,"[234.0, 432.0, 538.0, 644.0, 855.0]",0.309268
4,energy_delta_exp,"[112.0, 193.0, 226.0, 262.0, 302.0]",0.243142


In [54]:
a = [1,2]
a[2]

IndexError: list index out of range

In [ ]:
def energy_delta_squareM(signal, a, b, delta):
    return np.sum([((a*s - delta) ** 2) + b * s  for s in signal])

def energy_delta_squareD(signal, a, b, delta, c):
    energy = 0
    for i in np.arange(2,len(signal) - 1,1):
        energy = energy + (a*signal[i] - delta) ** 2 + b * signal[i] + c * [((signal[i] - signal[i-1]) + (signal[i-1] - signal[i-2])) / 2]  
        return energy

lines  = []
for b in np.arange(-10,10,0.1):
    for delta in np.arange(1100, 1180, 1):
        for c in np.arange(-1000, 1000, 1):
            H = []
            for curve in curves_std:
                h = energy_delta_squareD(curve[2:], 1, b, delta, c)
                H.append(h)
            mean = np.mean(H)
            mape = np.mean([abs(abs(mean - h)/mean) for h in H])
            lines.append([b, delta, H, mape])

m1 = pd.DataFrame(lines, columns = ['b', 'delta', 'H', 'MAPE'])
m1.sort_values(by ='MAPE')

In [44]:
a = [1,2,2,3,4]
a.index(2)

1

In [39]:
m1.to_csv("square_delta_otim.csv")

In [23]:
H = []
lines = []
for curve in curves_std:
    h = energy_delta_squareM(curve, 1, 0.3, 1175)
    H.append(h)
mean = np.mean(H)
mape = np.mean([abs(abs(mean - h)/mean) for h in H])
lines.append([0.3, 1175, H, mape])

lines

[[0.3,
  1175,
  [50806.6, 54412.0, 50385.5, 52765.49999999999, 51884.0],
  0.023638942938733584]]

In [ ]:
def best_model(signal):
    return np.sum([((s - 1175) ** 2) + 0.3 * s  for s in signal])

In [ ]:
exp_data['best_energy'] = exp_data['sinal'].apply(best_model)
exp_data

In [ ]:
exp_data

In [ ]:
tab_data['best_energy'] = tab_data['sinal'].apply(best_model)
tab_data

In [ ]:
from statsmodels.iolib.smpickle import load_pickle
sqrt_model = load_pickle("sqrt_fixed_alpha.pickle")

In [ ]:
f1 = np.arange(0.2, 2.6, 0.05)
f2 = np.sqrt(f1)
pf_continuos_x = [[1, f1[i], f2[i]] for i in np.arange(len(f1))]

ypred = sqrt_model.predict(pf_continuos_x)
est_curve_df = pd.DataFrame()
est_curve_df['alpha'] = f1
est_curve_df['final_temp'] = ypred
est_curve_df['final_temp_round'] = np.floor(ypred)

est_curves = []
for index, row in est_curve_df.iterrows():
    est_curves.append(segment_array(1160, row.final_temp, row.alpha))

est_curve_df['signal_curve'] = est_curves

In [ ]:
est_curve_df['best_energy'] = est_curve_df['signal_curve'].apply(best_model)
est_curve_df

In [ ]:
def stop_curve(signal, max_energy, energy_function):
    for i in range(0, len(signal)):
        current_energy = energy_function(signal[:i])
        if current_energy >= max_energy:
            break
    print(signal[0:i])
    print(signal[i])
    print(signal[i-20:i])
    return signal[0:i]

In [ ]:
exp_data['stop_curve'] = exp_data.sinal.apply(stop_curve, args=(64000,best_model))